<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_02/blob/main/RED_NEURONAL_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargar el dataset para revisar las primeras filas y obtener una descripción general de los datos
file_path = '/train_last_modified_v3.csv'
data = pd.read_csv(file_path)

# Mostrar las primeras filas y la información del dataset
print(data.head())
print(data.info())
print(data.describe())



In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Preprocesamiento de las características
scaler = StandardScaler()
X = scaler.fit_transform(data[['TMB', 'desayuno_carbs', 'desayuno_grasas', 'almuerzo_carbs', 'almuerzo_grasas', 'cena_carbs', 'cena_grasas']])

# Preparar etiquetas
tokenizer = Tokenizer()
# Asumiendo que cada columna de comidas es una lista convertida a cadena. Ajustar según los datos.
data['Desayuno'] = data['Desayuno'].apply(eval)  # Convierte de string de lista a lista real, si es necesario
tokenizer.fit_on_texts(data['Desayuno'].explode().astype(str))
vocab_size = len(tokenizer.word_index) + 1

Y_desayuno = tokenizer.texts_to_sequences(data['Desayuno'].astype(str))
Y_desayuno_padded = pad_sequences(Y_desayuno, padding='post', maxlen=10)  # Ajusta 'maxlen' según la longitud necesaria

# Dividir los datos en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_desayuno_padded, test_size=0.2, random_state=42)

# Imprimir tamaños para verificar
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (23517, 7)
Y_train shape: (23517, 10)
X_test shape: (5880, 7)
Y_test shape: (5880, 10)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, TimeDistributed, Dense

# Tamaño del vocabulario y longitud de secuencia
vocab_size = 16  # Asegúrate de que esto es correcto según tus IDs
sequence_length = 7  # Cambiar según la estructura de tus datos

# Reconstruir el modelo con la configuración correcta
model = Sequential([
    LSTM(100, input_shape=(sequence_length, 1), return_sequences=True),
    Dropout(0.5),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Preparar X_train y X_test para tener la forma correcta
X_train_lstm = X_train.reshape((X_train.shape[0], sequence_length, 1))
X_test_lstm = X_test.reshape((X_test.shape[0], sequence_length, 1))

# Asegurarse de que las formas de las etiquetas sean (None, 7, 16) ajustando si es necesario
Y_train_cat = Y_train_cat[:, :sequence_length, :]
Y_test_cat = Y_test_cat[:, :sequence_length, :]

# Verificar las formas finales
print("X_train_lstm shape:", X_train_lstm.shape)
print("Y_train_cat shape:", Y_train_cat.shape)
print("X_test_lstm shape:", X_test_lstm.shape)
print("Y_test_cat shape:", Y_test_cat.shape)

# Intentar entrenar el modelo nuevamente
history = model.fit(X_train_lstm, Y_train_cat, epochs=10, batch_size=32, validation_data=(X_test_lstm, Y_test_cat))


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 7, 100)            40800     
                                                                 
 dropout_4 (Dropout)         (None, 7, 100)            0         
                                                                 
 time_distributed_4 (TimeDi  (None, 7, 16)             1616      
 stributed)                                                      
                                                                 
Total params: 42416 (165.69 KB)
Trainable params: 42416 (165.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
X_train_lstm shape: (23517, 7, 1)
Y_train_cat shape: (23517, 7, 16)
X_test_lstm shape: (5880, 7, 1)
Y_test_cat shape: (5880, 7, 16)
Epoch 1/10
735/735 [==============================] - 11s 11ms/step - loss: 0.6773

In [14]:
model.save('fitLifeBuddy.keras')

In [15]:
from google.colab import files

# Asegúrate de haber guardado el modelo en formato .keras
model.save('fitLifeBuddy_IA.keras')

# Descargar el archivo al sistema local
files.download('fitLifeBuddy_IA.keras')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>